In [1]:
import os
import pandas as pd
import numpy as np
#!pip install crispr_chronos 
import chronos
import os
from taigapy import default_tc as tc
from taigapy import create_taiga_client_v3  
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()

2025-07-31 22:01:27.830256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 22:01:27.909523: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
chronos.__file__

'/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/__init__.py'

In [3]:
TAIGA_DATASET = "rasscreen07212025-0b0e"

# Load RAS data

In [4]:
RASFullCounts = tc.get(name=TAIGA_DATASET, file='RASFullCounts') 
RASFullGuideMap = tc.get(name=TAIGA_DATASET, file='RASFullGuideMap')
RASSequenceQCReport = tc.get(name=TAIGA_DATASET, file='RASSequenceQCReport')
#RASFullLfcSgrnaSeq = tc.download_to_cache(name='rasscreen02072125-6bdb', version=4, file='RASFullLfcSgrnaSeq')  # download_to_cache for raw
#RASFullNormCountsSgrnaSeq = tc.download_to_cache(name=TAIGA_DATASET, version=4, file='RASFullNormCountsSgrnaSeq')  # download_to_cache for raw
#RASGuideAgreementScore = tc.get(name=TAIGA_DATASET, version=4, file='RASGuideAgreementScore')
#RASRawCounts = tc.get(name=TAIGA_DATASET, version=4, file='RASRawCounts')
#RASRawCountsAllpDNAWells = tc.get(name=TAIGA_DATASET, version=4, file='RASRawCountsAllpDNAWells')
RASSequenceMap = tc.get(name=TAIGA_DATASET, file='RASSequenceMap')



No dataset version provided. Using version 15.
No dataset version provided. Using version 15.
No dataset version provided. Using version 15.
No dataset version provided. Using version 15.


In [5]:
# Library file
from taigapy import TaigaClient
tc = TaigaClient()
RASCommonEssentialControls = tc.get(name='ras-library-files-35fa',file='RasCombinatoryCommonEssentialControls')
RASNonessentialControls = tc.get(name='ras-library-files-35fa', file='RasCombinatoryNonEssentialControls')



No dataset version provided. Using version 15.
No dataset version provided. Using version 15.


In [6]:
sorted(RASFullCounts.columns)

['BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO205_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO678_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'HCT15_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'IPC298_250x-Cas9311-RAS-RepNA-P4(D21)',
 'IPC298_500x-Cas9311-RAS-RepNA-P4(D21)',
 'KP363T_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'PACADD188-Cas9VY-Ras-RepNA-P5(D25)',
 'PANC0203-Cas9VY-RAS-RepNA-P4(D21)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17)',
 'PANC1-Cas9VY-RAS-RepNA-P4(D21)',
 'RAS-pDNA',
 'SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'SNU410_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU478_250x-Cas9VY-RAS-RepNA-P4(D21)']

# Read counts matrix

In [7]:
readcounts = RASFullCounts.copy()
readcounts = readcounts.fillna(0)

# Sequence map

In [8]:
chronos_sequence_map = RASSequenceMap[['SequenceName', 'Screen', 'Days']].copy()
chronos_sequence_map['pDNA_batch'] = 'RAS'
chronos_sequence_map = chronos_sequence_map.rename(
        columns={
            'SequenceName': 'sequence_ID', 
            'Screen': 'cell_line_name', 
            'Days': 'days'
        }
)

In [9]:
# chronos_sequence_map.iloc[0,2]=0
chronos_sequence_map['days']=chronos_sequence_map['days'].fillna(21) 
chronos_sequence_map

,sequence_ID,cell_line_name,days,pDNA_batch
0,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PACADD188,25,RAS
1,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x_D17,17,RAS
2,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PANC0203_500x_D13,13,RAS
3,PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5),PANC0203_500x_D5,5,RAS
4,PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9),PANC0203_500x_D9,9,RAS
5,PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),PACADD165,21,RAS
6,KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T,21,RAS
7,SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),SNU1079,25,RAS
8,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678,25,RAS
9,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410,21,RAS


In [10]:
sorted(chronos_sequence_map['sequence_ID'])

['BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO205_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO678_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'HCT15_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'IPC298_250x-Cas9311-RAS-RepNA-P4(D21)',
 'IPC298_500x-Cas9311-RAS-RepNA-P4(D21)',
 'KP363T_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'PACADD188-Cas9VY-Ras-RepNA-P5(D25)',
 'PANC0203-Cas9VY-RAS-RepNA-P4(D21)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17)',
 'PANC1-Cas9VY-RAS-RepNA-P4(D21)',
 'RAS-pDNA',
 'SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'SNU410_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU478_250x-Cas9VY-RAS-RepNA-P4(D21)']

# Quality controls for the raw counts matrix

## Remove black list cell line

In [11]:
RASSequenceQCReport = RASSequenceQCReport.drop(columns=['Unnamed: 0'])
RASSequenceQCReport

,SequenceName,NNMD,NNMD_zscore,AvgRepresentations,MedianCommonEssentials,MedianNonessentials,MeanCommonEssentials,MeanNonessentials
0,PACADD188-Cas9VY-Ras-RepNA-P5(D25),-7.685994,-4.443070,396.337257,-1.725787,0.026069,-1.884848,-0.029968
1,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),-4.779225,-4.244235,705.255784,-0.842125,-0.014984,-1.051912,-0.002792
2,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),-3.220243,-2.991028,611.012732,-0.418619,-0.017874,-0.574775,-0.009965
3,PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5),-0.204920,-0.136493,1028.409739,0.005367,0.023803,-0.008677,0.012612
4,PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9),-1.215390,-1.303146,1015.674964,-0.148479,-0.036543,-0.219288,-0.035708
5,PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),-7.706549,-5.889288,1855.957375,-2.398313,0.015142,-2.600707,-0.010746
6,KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),-9.250705,-6.423726,5529.728610,-3.544754,-0.058496,-3.836968,-0.038307
7,SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),-0.560421,-0.483080,1092.360131,-0.181611,0.017745,-0.261583,-0.023804
8,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),-5.404073,-3.337582,744.848314,-1.160759,0.054450,-1.220242,-0.020304
9,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),-8.847861,-5.459631,897.384976,-2.134922,0.005124,-2.177861,-0.028335


In [12]:
black_list = [] 
readcounts = readcounts.loc[:, ~readcounts.columns.isin(black_list)]
chronos_sequence_map = chronos_sequence_map.loc[~chronos_sequence_map['sequence_ID'].isin(black_list)]

In [13]:
# Remove low NNMD
black_list = RASSequenceQCReport.loc[RASSequenceQCReport['NNMD'] > -1.25, ].SequenceName.to_list()
black_list

['PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9)',
 'SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25)']

In [14]:
readcounts

,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5),PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9),PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),...,HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),SNU410_500x-Cas9VY-RAS-RepNA-P4(D21),SNU410_250x-Cas9VY-RAS-RepNA-P4(D21),IPC298_500x-Cas9311-RAS-RepNA-P4(D21),IPC298_250x-Cas9311-RAS-RepNA-P4(D21),PANC1-Cas9VY-RAS-RepNA-P4(D21),PANC0203-Cas9VY-RAS-RepNA-P4(D21),RAS-pDNA
AAVS1_L5_ABCB7_R4,1.0,185.0,621.0,524.0,953.0,943.0,23.0,5595.0,321.0,114.0,...,279.0,431.0,440.0,275.0,57.0,1.0,0.0,142.0,260.0,2799.0
AAVS1_L5_ABCB7_R6,94.0,258.0,508.0,396.0,826.0,143.0,0.0,5136.0,515.0,111.0,...,21.0,428.0,263.0,635.0,30.0,0.0,0.0,182.0,468.0,2236.0
AAVS1_L5_ABCF1_R2,28.0,132.0,332.0,369.0,588.0,293.0,195.0,1745.0,238.0,67.0,...,24.0,336.0,221.0,746.0,166.0,54.0,13.0,283.0,133.0,2033.0
AAVS1_L5_ABCF1_R4,98.0,111.0,182.0,324.0,445.0,363.0,36.0,1705.0,410.0,329.0,...,701.0,491.0,165.0,963.0,147.0,510.0,64.0,261.0,284.0,1951.0
AAVS1_L5_ABL1_R2,409.0,450.0,644.0,501.0,1071.0,1104.0,926.0,3581.0,886.0,876.0,...,2694.0,1185.0,957.0,2533.0,1396.0,671.0,325.0,402.0,1542.0,2996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAZ_L3_YWHAG_R6,582.0,534.0,754.0,599.0,926.0,1215.0,1774.0,2969.0,1066.0,691.0,...,169.0,938.0,429.0,3327.0,2696.0,79.0,382.0,368.0,1158.0,2988.0
YY1_L1_AAVS1_R2,39.0,117.0,223.0,314.0,510.0,719.0,336.0,7338.0,126.0,91.0,...,281.0,243.0,296.0,525.0,377.0,124.0,20.0,30.0,304.0,2470.0
YY1_L5_AAVS1_R2,93.0,159.0,373.0,392.0,772.0,533.0,1643.0,706.0,54.0,901.0,...,117.0,532.0,575.0,1017.0,392.0,101.0,65.0,117.0,313.0,2988.0
ZNF143_L1_AAVS1_R2,218.0,377.0,707.0,561.0,1020.0,663.0,2309.0,6936.0,1522.0,426.0,...,328.0,888.0,1219.0,1090.0,878.0,129.0,134.0,489.0,829.0,4121.0


In [15]:
chronos_sequence_map

,sequence_ID,cell_line_name,days,pDNA_batch
0,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PACADD188,25,RAS
1,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x_D17,17,RAS
2,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PANC0203_500x_D13,13,RAS
3,PANC0203_500x-Cas9VY-RAS-RepNA-P4(D5),PANC0203_500x_D5,5,RAS
4,PANC0203_500x-Cas9VY-RAS-RepNA-P4(D9),PANC0203_500x_D9,9,RAS
5,PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),PACADD165,21,RAS
6,KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T,21,RAS
7,SNU1079_250x-Cas9VY-RAS-RepNA-P5(D25),SNU1079,25,RAS
8,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678,25,RAS
9,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410,21,RAS


In [16]:
chronos_sequence_map = chronos_sequence_map.loc[~chronos_sequence_map['sequence_ID'].isin(black_list)]
readcounts = readcounts.loc[:,chronos_sequence_map['sequence_ID'] ]


In [17]:
readcounts

,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),...,HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),SNU410_500x-Cas9VY-RAS-RepNA-P4(D21),SNU410_250x-Cas9VY-RAS-RepNA-P4(D21),IPC298_500x-Cas9311-RAS-RepNA-P4(D21),IPC298_250x-Cas9311-RAS-RepNA-P4(D21),PANC1-Cas9VY-RAS-RepNA-P4(D21),PANC0203-Cas9VY-RAS-RepNA-P4(D21),RAS-pDNA
AAVS1_L5_ABCB7_R4,1.0,185.0,621.0,943.0,23.0,321.0,114.0,663.0,0.0,79.0,...,279.0,431.0,440.0,275.0,57.0,1.0,0.0,142.0,260.0,2799.0
AAVS1_L5_ABCB7_R6,94.0,258.0,508.0,143.0,0.0,515.0,111.0,194.0,1.0,34.0,...,21.0,428.0,263.0,635.0,30.0,0.0,0.0,182.0,468.0,2236.0
AAVS1_L5_ABCF1_R2,28.0,132.0,332.0,293.0,195.0,238.0,67.0,127.0,70.0,54.0,...,24.0,336.0,221.0,746.0,166.0,54.0,13.0,283.0,133.0,2033.0
AAVS1_L5_ABCF1_R4,98.0,111.0,182.0,363.0,36.0,410.0,329.0,251.0,135.0,185.0,...,701.0,491.0,165.0,963.0,147.0,510.0,64.0,261.0,284.0,1951.0
AAVS1_L5_ABL1_R2,409.0,450.0,644.0,1104.0,926.0,886.0,876.0,1051.0,451.0,1256.0,...,2694.0,1185.0,957.0,2533.0,1396.0,671.0,325.0,402.0,1542.0,2996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAZ_L3_YWHAG_R6,582.0,534.0,754.0,1215.0,1774.0,1066.0,691.0,436.0,525.0,654.0,...,169.0,938.0,429.0,3327.0,2696.0,79.0,382.0,368.0,1158.0,2988.0
YY1_L1_AAVS1_R2,39.0,117.0,223.0,719.0,336.0,126.0,91.0,239.0,19.0,140.0,...,281.0,243.0,296.0,525.0,377.0,124.0,20.0,30.0,304.0,2470.0
YY1_L5_AAVS1_R2,93.0,159.0,373.0,533.0,1643.0,54.0,901.0,331.0,323.0,80.0,...,117.0,532.0,575.0,1017.0,392.0,101.0,65.0,117.0,313.0,2988.0
ZNF143_L1_AAVS1_R2,218.0,377.0,707.0,663.0,2309.0,1522.0,426.0,976.0,288.0,1143.0,...,328.0,888.0,1219.0,1090.0,878.0,129.0,134.0,489.0,829.0,4121.0


In [18]:
chronos_sequence_map["cell_line_name"] = chronos_sequence_map["sequence_ID"]
chronos_sequence_map

/tmp/ipykernel_23735/2914751146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chronos_sequence_map["cell_line_name"] = chronos_sequence_map["sequence_ID"]


,sequence_ID,cell_line_name,days,pDNA_batch
0,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PACADD188-Cas9VY-Ras-RepNA-P5(D25),25,RAS
1,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),17,RAS
2,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),13,RAS
5,PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
6,KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
8,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),25,RAS
9,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
10,BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
11,SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
12,HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS


In [19]:
guide_map = RASFullGuideMap[['GuideLabel', 'GuideTargetSymbol']].copy()
guide_map = guide_map.rename(columns=({'GuideLabel': 'sgrna', 'GuideTargetSymbol': 'gene'}))
good_guides = RASFullGuideMap.loc[RASFullGuideMap['DropReasonsRAS'].isnull(),'GuideLabel']
guide_map = guide_map.loc[guide_map['sgrna'].isin(good_guides)]

In [20]:
readcounts

,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),...,HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),SNU410_500x-Cas9VY-RAS-RepNA-P4(D21),SNU410_250x-Cas9VY-RAS-RepNA-P4(D21),IPC298_500x-Cas9311-RAS-RepNA-P4(D21),IPC298_250x-Cas9311-RAS-RepNA-P4(D21),PANC1-Cas9VY-RAS-RepNA-P4(D21),PANC0203-Cas9VY-RAS-RepNA-P4(D21),RAS-pDNA
AAVS1_L5_ABCB7_R4,1.0,185.0,621.0,943.0,23.0,321.0,114.0,663.0,0.0,79.0,...,279.0,431.0,440.0,275.0,57.0,1.0,0.0,142.0,260.0,2799.0
AAVS1_L5_ABCB7_R6,94.0,258.0,508.0,143.0,0.0,515.0,111.0,194.0,1.0,34.0,...,21.0,428.0,263.0,635.0,30.0,0.0,0.0,182.0,468.0,2236.0
AAVS1_L5_ABCF1_R2,28.0,132.0,332.0,293.0,195.0,238.0,67.0,127.0,70.0,54.0,...,24.0,336.0,221.0,746.0,166.0,54.0,13.0,283.0,133.0,2033.0
AAVS1_L5_ABCF1_R4,98.0,111.0,182.0,363.0,36.0,410.0,329.0,251.0,135.0,185.0,...,701.0,491.0,165.0,963.0,147.0,510.0,64.0,261.0,284.0,1951.0
AAVS1_L5_ABL1_R2,409.0,450.0,644.0,1104.0,926.0,886.0,876.0,1051.0,451.0,1256.0,...,2694.0,1185.0,957.0,2533.0,1396.0,671.0,325.0,402.0,1542.0,2996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAZ_L3_YWHAG_R6,582.0,534.0,754.0,1215.0,1774.0,1066.0,691.0,436.0,525.0,654.0,...,169.0,938.0,429.0,3327.0,2696.0,79.0,382.0,368.0,1158.0,2988.0
YY1_L1_AAVS1_R2,39.0,117.0,223.0,719.0,336.0,126.0,91.0,239.0,19.0,140.0,...,281.0,243.0,296.0,525.0,377.0,124.0,20.0,30.0,304.0,2470.0
YY1_L5_AAVS1_R2,93.0,159.0,373.0,533.0,1643.0,54.0,901.0,331.0,323.0,80.0,...,117.0,532.0,575.0,1017.0,392.0,101.0,65.0,117.0,313.0,2988.0
ZNF143_L1_AAVS1_R2,218.0,377.0,707.0,663.0,2309.0,1522.0,426.0,976.0,288.0,1143.0,...,328.0,888.0,1219.0,1090.0,878.0,129.0,134.0,489.0,829.0,4121.0


In [21]:
readcounts = readcounts.loc[guide_map['sgrna']]


In [22]:
#--- Transpose read counts matrix into cell line by sgRNA
readcounts = readcounts.T


# Chronos training for parental

In [24]:
index_safeharbor_sgrna = guide_map['gene'].isin(['AAVS1', 'chr2'])
index_hart_nonessential_sgrna = guide_map['gene'].isin(RASNonessentialControls['GuideTargetSymbol'])


In [25]:
import chronos
chronos.__file__

'/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/__init__.py'

In [33]:
chronos_sequence_map.loc[chronos_sequence_map["sequence_ID"] == "RAS-pDNA", "cell_line_name"] = "pDNA"
chronos_sequence_map

,sequence_ID,cell_line_name,days,pDNA_batch
0,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PACADD188-Cas9VY-Ras-RepNA-P5(D25),25,RAS
1,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),17,RAS
2,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),13,RAS
5,PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
6,KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
8,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),25,RAS
9,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
10,BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
11,SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
12,HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS


In [34]:
#--- Remove outgrowths by chronos
chronos.nan_outgrowths(readcounts,chronos_sequence_map, guide_map)

calculating LFC
finding maximum LFC calls
filtering


/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:418: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  max_lfc = lfc.groupby(ggtemp, axis=1).max()
/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:433: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  max_row_2nd_column = lfc_filtered.T.groupby(ggtemp, axis=0).agg(second_highest).T


finding second highest LFC calls
finding sequences and guides with outgrowth


/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:444: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  masked = masked.reindex(index=lfc_filtered.index, columns=ggtemp.values).fillna(False)
/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:449: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = mask.reindex(index=readcounts.index, columns=readcounts.columns).fillna(False).astype(bool)


NAing 93 readcounts (0.00005 of total)


In [35]:
#--- It is much better to use cutting than noncutting controls, and as many as possible.
negative_control_sgrnas = guide_map.loc[index_hart_nonessential_sgrna,]
negative_control_sgrnas = negative_control_sgrnas['sgrna'].to_list()


In [36]:
chronos_sequence_map['days']=[int(i) for i in chronos_sequence_map['days']]
chronos_sequence_map

/tmp/ipykernel_23735/1888508864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chronos_sequence_map['days']=[int(i) for i in chronos_sequence_map['days']]


,sequence_ID,cell_line_name,days,pDNA_batch
0,PACADD188-Cas9VY-Ras-RepNA-P5(D25),PACADD188-Cas9VY-Ras-RepNA-P5(D25),25,RAS
1,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),17,RAS
2,PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),13,RAS
5,PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
6,KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
8,COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),25,RAS
9,SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
10,BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
11,SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS
12,HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),21,RAS


In [37]:
guide_map

,sgrna,gene
0,AAVS1_L5_ABCB7_R4,ABCB7
1,AAVS1_L5_ABCB7_R6,ABCB7
2,AAVS1_L5_ABCF1_R2,ABCF1
3,AAVS1_L5_ABCF1_R4,ABCF1
4,AAVS1_L5_ABL1_R2,ABL1
...,...,...
84195,YWHAZ_L3_YWHAG_R6,YWHAG_YWHAZ
84196,YY1_L1_AAVS1_R2,YY1
84197,YY1_L5_AAVS1_R2,YY1
84198,ZNF143_L1_AAVS1_R2,ZNF143


In [38]:
negative_control_sgrnas

['AAVS1_L5_ACTL7A_R2',
 'AAVS1_L5_ACTL7A_R4',
 'AAVS1_L5_ACTL7B_R4',
 'AAVS1_L5_ACTL7B_R6',
 'AAVS1_L5_ACTL9_R4',
 'AAVS1_L5_ACTL9_R6',
 'AAVS1_L5_ADAM18_R2',
 'AAVS1_L5_ADAM18_R6',
 'AAVS1_L5_ADAM20_R2',
 'AAVS1_L5_ADAM20_R6',
 'AAVS1_L5_ADAM30_R4',
 'AAVS1_L5_ADAM30_R6',
 'AAVS1_L5_AICDA_R2',
 'AAVS1_L5_AICDA_R4',
 'AAVS1_L5_ALPG_R2',
 'AAVS1_L5_ALPG_R6',
 'AAVS1_L5_ALX3_R4',
 'AAVS1_L5_ALX3_R6',
 'AAVS1_L5_ASB17_R4',
 'AAVS1_L5_ASB17_R6',
 'AAVS1_L5_ATP4B_R4',
 'AAVS1_L5_ATP4B_R6',
 'AAVS1_L5_ATP6V1G3_R2',
 'AAVS1_L5_ATP6V1G3_R4',
 'AAVS1_L5_B3GNT6_R4',
 'AAVS1_L5_B3GNT6_R6',
 'AAVS1_L5_BPIFA3_R2',
 'AAVS1_L5_BPIFA3_R4',
 'AAVS1_L5_CABP2_R2',
 'AAVS1_L5_CABP2_R4',
 'AAVS1_L5_CABP5_R2',
 'AAVS1_L5_CABP5_R4',
 'AAVS1_L5_CCKAR_R2',
 'AAVS1_L5_CCKAR_R6',
 'AAVS1_L5_CCT8L2_R4',
 'AAVS1_L5_CCT8L2_R6',
 'AAVS1_L5_CEACAM7_R2',
 'AAVS1_L5_CEACAM7_R6',
 'AAVS1_L5_CELA2A_R2',
 'AAVS1_L5_CELA2A_R4',
 'AAVS1_L5_CELA3A_R4',
 'AAVS1_L5_CELA3A_R6',
 'AAVS1_L5_CELA3B_R2',
 'AAVS1_L5_CELA3B_R4',
 'AA

In [39]:
readcounts

,AAVS1_L5_ABCB7_R4,AAVS1_L5_ABCB7_R6,AAVS1_L5_ABCF1_R2,AAVS1_L5_ABCF1_R4,AAVS1_L5_ABL1_R2,AAVS1_L5_ABL1_R6,AAVS1_L5_ACTL7A_R2,AAVS1_L5_ACTL7A_R4,AAVS1_L5_ACTL7B_R4,AAVS1_L5_ACTL7B_R6,...,YWHAZ_L3_YAP1_R2,YWHAZ_L3_YAP1_R4,YWHAZ_L3_YWHAE_R4,YWHAZ_L3_YWHAE_R6,YWHAZ_L3_YWHAG_R4,YWHAZ_L3_YWHAG_R6,YY1_L1_AAVS1_R2,YY1_L5_AAVS1_R2,ZNF143_L1_AAVS1_R2,ZNF143_L3_AAVS1_R2
PACADD188-Cas9VY-Ras-RepNA-P5(D25),1.0,94.0,28.0,98.0,409.0,166.0,398.0,249.0,535.0,585.0,...,234.0,965.0,1324.0,861.0,256.0,582.0,39.0,93.0,218.0,122.0
PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17),185.0,258.0,132.0,111.0,450.0,382.0,437.0,289.0,503.0,250.0,...,552.0,827.0,1620.0,1489.0,519.0,534.0,117.0,159.0,377.0,268.0
PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13),621.0,508.0,332.0,182.0,644.0,686.0,709.0,470.0,615.0,393.0,...,847.0,1251.0,2256.0,1879.0,928.0,754.0,223.0,373.0,707.0,640.0
PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21),943.0,143.0,293.0,363.0,1104.0,595.0,920.0,526.0,1363.0,1139.0,...,593.0,788.0,643.0,2028.0,1241.0,1215.0,719.0,533.0,663.0,1522.0
KP363T_250x-Cas9VY-RAS-RepNA-P4(D21),23.0,0.0,195.0,36.0,926.0,808.0,2685.0,1929.0,2681.0,595.0,...,2605.0,1982.0,6671.0,6625.0,1198.0,1774.0,336.0,1643.0,2309.0,259.0
COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),321.0,515.0,238.0,410.0,886.0,416.0,2124.0,931.0,384.0,608.0,...,819.0,1742.0,4584.0,4445.0,1262.0,1066.0,126.0,54.0,1522.0,1039.0
SNU410_125x-Cas9VY-RAS-RepNA-P4(D21),114.0,111.0,67.0,329.0,876.0,516.0,521.0,411.0,325.0,245.0,...,862.0,947.0,1211.0,3091.0,689.0,691.0,91.0,901.0,426.0,820.0
BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),663.0,194.0,127.0,251.0,1051.0,555.0,740.0,837.0,704.0,427.0,...,675.0,1143.0,1586.0,1809.0,924.0,436.0,239.0,331.0,976.0,1025.0
SNU478_250x-Cas9VY-RAS-RepNA-P4(D21),0.0,1.0,70.0,135.0,451.0,586.0,939.0,319.0,454.0,376.0,...,776.0,456.0,1242.0,590.0,687.0,525.0,19.0,323.0,288.0,595.0
HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),79.0,34.0,54.0,185.0,1256.0,516.0,1149.0,1058.0,912.0,478.0,...,1026.0,1843.0,1876.0,2093.0,921.0,654.0,140.0,80.0,1143.0,552.0


In [40]:

model = chronos.Chronos(
    sequence_map={"RASV2": chronos_sequence_map},
    guide_gene_map={"RASV2": guide_map},
    readcounts={"RASV2": readcounts},
    negative_control_sgrnas={'RASV2': negative_control_sgrnas}
)

# model = chronos.Chronos(
#     sequence_map={"RASV2": chronos_sequence_map},
#     guide_gene_map={"RASV2": guide_map},
#     readcounts={"RASV2": readcounts},
#     negative_control_sgrnas={'RASV2': negative_control_sgrnas},
#     to_normalize_readcounts=True
# )



normalizing readcounts


Finding all unique guides and genes
found 84197 unique guides and 10754 unique genes in RASV2
found 84197 unique guides and 10754 unique genes overall

finding guide-gene mapping indices

finding all unique sequenced replicates, cell lines, and pDNA batches
found 20 unique sequences (excluding pDNA) and 20 unique cell lines in RASV2
found 20 unique replicates and 20 unique cell lines overall

finding replicate-cell line mappings indices

finding replicate-pDNA mappings indices


assigning float constants
Estimating or aligning variances
	Estimating excess variance (alpha) for RASV2
Between 1 (batch=Index(['RASV2_RAS'], dtype='object')) and 1 (batch=Index(['RASV2_RAS'], dtype='object')) negative control sgRNAs were found to be systematically over- or under-represented in the screens and excluded.
Creating excess variance tensors
	Created excess variance tensor for RASV2 with shape [20, 1]
initializing graph

building gene effect mask

building doubling vectors
m

2025-07-31 22:06:36.107775: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:1196: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  key: readcounts[key]\
2025-07-31 22:06:36.993441: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


	built normalized timepoints for RASV2 with shape (20, 84197) (replicates X guides)

building t0 reads


Building variables

building t0 reads estimate
made t0 batch with shape (1, 84197) for RASV2
building guide efficacy
built guide efficacy: shape [1, 84198]
building growth rate
built growth rate: shape {'RASV2': [20, 1]}

building cell line efficacy
built cell line efficacy: shapes {'RASV2': [20, 1]}
building screen delay
built screen delay
building gene effect
built core gene effect: 20 cell lines by 10754 genes


Connecting graph nodes in model

building effective days
built effective days, shapes {'RASV2': [20, 10754]}

building gene effect growth graph nodes
built gene effect growth graph nodes, shapes {'RASV2': [20, 10754]}

building combined efficacy
built combined efficacy, shape {'RASV2': [20, 84197]}
built expanded combined efficacy, shapes {'RASV2': [20, 84197]}

building growth estimates of edited cells and overall estimates of fold change in guide abundance
built growth 

2025-07-31 22:06:39.781541: W tensorflow/c/c_api.cc:291] Operation '{name:'excess_variance/RASV2/Assign' id:6 op device:{requested: '', assigned: ''} def:{{{node excess_variance/RASV2/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_DOUBLE, validate_shape=false](excess_variance/RASV2, excess_variance/RASV2/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


estimating initial screen efficacy and gene effect
	 RASV2


/home/ubuntu/miniconda3/envs/paralog/lib/python3.10/site-packages/chronos/model.py:1772: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  mean_fold_change = fold_change\


	verifying graph integrity
verifying user inputs
verifying variables
verifying calculated terms
	RASV2 _gene_effect
	RASV2 _selected_efficacies
	RASV2_predicted_readcounts_unscaled
	RASV2 _predicted_readcounts
	RASV2 _normalized_readcounts
	RASV2 _cost_presum
sess run
	RASV2 _cost
	RASV2 _full_costs
ready to train


In [41]:
model.train(1001, report_freq=50, burn_in_period=50, ge_only=0)

NB2 cost 0.31883103067787355
Full cost 0.3562863737975988
relative_growth_rate
	RASV2 max 1.021, min 0.97576
mean guide efficacy 0.9920726526826145
t0_offset SD: [('RASV2', 7.239473450212771e-05)]

gene mean -0.19876158914198364
SD of gene means 0.3105396702155306
Mean of gene SDs 0.28590584353736526



51 epochs trained, time taken 0:00:48, projected remaining 0:14:59
NB2 cost 0.21666481321974557
Full cost 0.25575355813169276
relative_growth_rate
	RASV2 max 1.463, min 0.72777
mean guide efficacy 0.9298642756752105
t0_offset SD: [('RASV2', 0.14938664390429113)]

gene mean -0.04187417290369461
SD of gene means 0.4017072037006598
Mean of gene SDs 0.30100334971365517



101 epochs trained, time taken 0:01:35, projected remaining 0:14:06
NB2 cost 0.2083861185969277
Full cost 0.24004424693513937
relative_growth_rate
	RASV2 max 1.298, min 0.72530
mean guide efficacy 0.8819378840255883
t0_offset SD: [('RASV2', 0.12729977745456483)]

gene mean 0.04534918735854306
SD of gene means 0.40973204393

In [33]:
model

Chronos model with libraries ['RASV2'], 14 total cell lines, 10741 total genes, trained for 1001 epochs

In [42]:
BASEDIR = "/home/ubuntu/paralog/ras_screens/analysis/RASScreen07212025/"
model.save( os.path.join(BASEDIR, "0721_chronos"), overwrite=True)

In [35]:
#model = chronos.load_saved_model(os.path.join(BASEDIR, "chronos"))

In [93]:
gene_effect = model.gene_effect

2025-07-31 22:56:59.709147: W tensorflow/c/c_api.cc:291] Operation '{name:'Variable_1/Adam_1/Assign' id:3278 op device:{requested: '', assigned: ''} def:{{{node Variable_1/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_DOUBLE, validate_shape=false](Variable_1/Adam_1, Variable_1/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


# Copy number correction

In [94]:
from taigapy import create_taiga_client_v3  
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()

In [95]:
#OmicsCNGeneFullProfileRASV2 = chronos.read_hdf5("../data/RASScreen05132024/OmicsCNGeneFullProfileRASV2.hdf5")
OmicsCNGeneFullProfileRASV2 = tc.get(name=TAIGA_DATASET, file='OmicsCNGeneFullProfileRASV2')  # download_to_cache for raw
OmicsCNGeneFullProfileRASV2 = OmicsCNGeneFullProfileRASV2[gene_effect.columns]

RASScreenMap = RASSequenceMap[['SequenceName', 'CellLine', 'ModelID']].drop_duplicates().set_index('SequenceName')
RASScreenMap = RASScreenMap.loc[gene_effect.index]
RASScreenMap.loc[RASScreenMap['ModelID'].isna(), 'ModelID'] = RASScreenMap.loc[RASScreenMap['ModelID'].isna(), ].index



No dataset version provided. Using version 17.


In [96]:
copy_number_profile = ( 
    RASScreenMap.
        merge(OmicsCNGeneFullProfileRASV2, how = 'left', left_on = 'ModelID', right_index=True).
        drop(['CellLine'], axis=1).
        drop(['ModelID'], axis=1)
)

In [97]:
copy_number_profile

,ABCB7,ABCF1,ABL1,ACTL7A,ACTL7B,ACTL9,ADAM18,ADAM20,ADAM30,AFG3L2,...,YAP1_YWHAG,YAP1_YWHAZ,YWHAE,YWHAE_YWHAG,YWHAE_YWHAZ,YWHAG,YWHAG_YWHAZ,YWHAZ,YY1,ZNF143
cell_line_name,,,,,,,,,,,,,,,,,,,,,
BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21),0.929710,0.870387,1.236430,1.419715,1.419715,0.827303,1.323272,1.188036,1.076562,0.934473,...,2.900377,2.781340,0.818810,2.213306,2.094268,1.394496,2.669955,1.275459,1.201395,0.922648
BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21),0.929710,0.870387,1.236430,1.419715,1.419715,0.827303,1.323272,1.188036,1.076562,0.934473,...,2.900377,2.781340,0.818810,2.213306,2.094268,1.394496,2.669955,1.275459,1.201395,0.922648
BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21),0.929710,0.870387,1.236430,1.419715,1.419715,0.827303,1.323272,1.188036,1.076562,0.934473,...,2.900377,2.781340,0.818810,2.213306,2.094268,1.394496,2.669955,1.275459,1.201395,0.922648
COLO205_250x-Cas9VY-RAS-RepNA-P4(D21),0.547492,0.985400,1.150980,1.150980,1.150980,0.939516,0.589237,0.970185,0.624576,0.630750,...,1.824509,2.011607,1.051698,1.988558,2.175655,0.936860,2.060817,1.123957,0.970185,0.935269
COLO678_250x-Cas9VY-RAS-RepNA-P5(D25),0.814662,0.885113,0.900764,1.043124,1.043124,0.888962,0.965243,0.930270,1.016781,0.974481,...,3.092828,3.015285,0.882548,1.947406,1.869864,1.064858,2.052173,0.987315,0.917738,0.925982
HCT15_250x-Cas9VY-RAS-RepNA-P4(D21),0.488206,0.995882,0.994496,0.994496,0.994496,0.958509,0.992196,1.011965,0.998813,0.997424,...,1.997288,1.985410,0.997680,2.001754,1.989876,1.004074,1.996270,0.992196,0.981362,0.993214
HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21),0.766498,0.900468,0.879374,0.879374,0.879374,0.916412,1.010747,1.568399,1.029765,0.859138,...,2.489964,2.220552,1.032255,2.335466,2.066054,1.303211,2.337010,1.033799,1.568399,0.790848
IPC298_250x-Cas9311-RAS-RepNA-P4(D21),0.647532,1.113070,0.846364,0.949177,0.949177,1.015841,1.171571,0.860844,3.368852,1.567224,...,2.536866,2.154006,0.548946,2.105479,1.722619,1.556533,2.730205,1.173673,0.854500,0.930137
IPC298_500x-Cas9311-RAS-RepNA-P4(D21),0.647532,1.113070,0.846364,0.949177,0.949177,1.015841,1.171571,0.860844,3.368852,1.567224,...,2.536866,2.154006,0.548946,2.105479,1.722619,1.556533,2.730205,1.173673,0.854500,0.930137


In [98]:
sorted(copy_number_profile.index)

['BXPC3_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'BXPC3_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO205_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'COLO678_250x-Cas9VY-RAS-RepNA-P5(D25)',
 'HCT15_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'HPAFII_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'IPC298_250x-Cas9311-RAS-RepNA-P4(D21)',
 'IPC298_500x-Cas9311-RAS-RepNA-P4(D21)',
 'KP363T_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'PACADD165_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'PACADD188-Cas9VY-Ras-RepNA-P5(D25)',
 'PANC0203-Cas9VY-RAS-RepNA-P4(D21)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P5(D13)',
 'PANC0203_500x-Cas9VY-RAS-RepNA-P5(D17)',
 'PANC1-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_125x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_250x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU410_500x-Cas9VY-RAS-RepNA-P4(D21)',
 'SNU478_250x-Cas9VY-RAS-RepNA-P4(D21)']

In [99]:
copy_number_profile = copy_number_profile.fillna(1)
copy_number_profile = copy_number_profile.loc[gene_effect.index, gene_effect.columns ]
out = chronos.alternate_CN(gene_effect, copy_number_profile)



Fitting cell line group 1 of 1
finding low CN gene effect shifts
smoothing and interpolating cutting toxicity for all genes
constructed spline matrix of shape 215080, 105
	cost: 0.11691047952679
	cost: 0.11101473252360555
	cost: 0.10942436353329398
	cost: 0.10881747445064274
	cost: 0.10850044794239225
	cost: 0.10828601104211406
generating matrix


In [100]:
gene_effect_target_corrected = out[0]
shifts_target = out[1]

In [101]:
gene_effect_final = gene_effect_target_corrected.copy()

In [102]:
#gene_effect.columns  = RASTargetMap['GuideTargetSymbol']
gene_effect_final -= gene_effect_final.reindex(columns=RASNonessentialControls['GuideTargetSymbol'].to_list()).median(axis=1).median()
gene_effect_final /= gene_effect_final.reindex(columns= RASCommonEssentialControls['GuideTargetSymbol'].to_list()).median(axis=1).abs().median()



In [103]:
gene_effect_screen = gene_effect_final.copy()
gene_effect_screen = gene_effect_screen.merge(RASSequenceMap[['SequenceName', 'Screen']].set_index('SequenceName'), left_index=True, right_index=True, how='left')
gene_effect_screen = gene_effect_screen.groupby("Screen").median()
gene_effect_screen

,ABCB7,ABCF1,ABL1,ACTL7A,ACTL7B,ACTL9,ADAM18,ADAM20,ADAM30,AFG3L2,...,YAP1_YWHAG,YAP1_YWHAZ,YWHAE,YWHAE_YWHAG,YWHAE_YWHAZ,YWHAG,YWHAG_YWHAZ,YWHAZ,YY1,ZNF143
Screen,,,,,,,,,,,,,,,,,,,,,
BXPC3,-1.180169,-1.220149,-0.224970,-0.084473,0.011948,-0.073987,-0.003601,-0.248391,-0.235217,-0.449453,...,-0.121955,-0.102114,0.129657,0.152929,-0.271025,-0.173431,-0.301692,-0.191111,-1.294156,-0.012417
COLO205,-0.718061,-0.880274,-0.062943,0.023210,-0.048788,-0.051824,-0.195366,-0.026295,-0.087829,-0.622326,...,-0.121210,-0.604548,0.078998,0.142012,-0.480383,-0.047827,-0.859941,-0.437065,-0.794764,0.074005
COLO678,-0.984268,-0.460997,-0.187423,0.103760,-0.314502,0.252181,0.092994,0.089517,0.042634,-0.507663,...,-0.431324,-0.398691,0.146501,0.494367,0.399418,-0.093901,0.185038,0.004712,-1.612199,-0.059100
HCT15,-2.067564,-1.692507,-0.031890,0.100401,0.068557,-0.068687,0.238079,-0.121337,0.213761,-0.576173,...,-0.516873,-0.269154,0.187629,0.310857,-0.087794,0.015064,-0.471259,-0.141680,-1.489052,-0.123620
HPAFII,-1.101443,-0.638445,0.133116,-0.078443,0.062730,0.168445,-0.067225,-0.119957,-0.144884,-0.860185,...,0.103994,-0.543388,-0.021062,0.007701,-0.990446,-0.208105,-1.069454,-0.486183,-0.793323,-0.074650
IPC298,-1.355939,-1.076866,0.005116,-0.180341,0.051066,-0.152538,-0.311513,-0.078241,-0.440655,-0.528256,...,-0.053753,-0.277030,-0.225452,-0.128495,-0.970008,-0.042486,-0.453505,-0.315810,-0.710198,-0.185330
KP363T,-1.090039,-1.368010,-0.280611,-0.206993,0.054511,0.043066,-0.308823,0.218928,-0.292706,-0.362938,...,0.453468,0.234958,0.250957,0.363380,0.360987,-0.038199,-0.145377,-0.114315,-0.638440,-0.461291
PACADD165,-0.959826,-1.354044,-0.352820,-0.449311,0.187461,0.041026,-0.262529,-0.146144,-0.072290,-0.674041,...,-0.144478,-0.369666,0.089084,0.176295,-0.247866,-0.016899,-0.166290,-0.148120,-0.310542,-0.061025
PACADD188,-1.569134,-0.911991,-0.103256,-0.047919,0.164574,0.022319,-0.161648,-0.125220,-0.186660,-0.169646,...,-0.088983,-0.234243,0.170273,0.197351,-0.020886,-0.012996,-0.185185,-0.265480,-0.733314,-0.302735


In [104]:
chronos.write_hdf5(gene_effect_final, "0721RASGeneEffect.hdf5")
chronos.write_hdf5(gene_effect, "0721RASGeneEffectUncorrected.hdf5")
chronos.write_hdf5(gene_effect_screen, "0721RASGeneEffectScreenLevel.hdf5")

# Uploading chronos gene effect

In [105]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset(TAIGA_DATASET, 
                            "Uploading Chronos Gene Effect and the copy number profile for chronos correction", 
                            additions=[
    UploadedFile('RASGeneEffectUncorrected', 
                 local_path=os.path.join("0721RASGeneEffectUncorrected.hdf5"), 
                 format=LocalFormat.HDF5_MATRIX),
    UploadedFile('RASGeneEffect', 
                 local_path=os.path.join("0721RASGeneEffect.hdf5"), 
                 format=LocalFormat.HDF5_MATRIX),
    UploadedFile('RASGeneEffectScreenLevel', 
                 local_path=os.path.join("0721RASGeneEffectScreenLevel.hdf5"), 
                 format=LocalFormat.HDF5_MATRIX)
    ]
)

Created temporary upload session 43a1dd7110e2434eb6fac60e5df37dd1
Uploading 0721RASGeneEffectUncorrected.hdf5 to S3
Finished uploading 0721RASGeneEffectUncorrected.hdf5 to S3
Completed uploading 0721RASGeneEffectUncorrected.hdf5 to S3
Added 0721RASGeneEffectUncorrected.hdf5 to upload session 43a1dd7110e2434eb6fac60e5df37dd1
Uploading 0721RASGeneEffect.hdf5 to S3
Finished uploading 0721RASGeneEffect.hdf5 to S3
Completed uploading 0721RASGeneEffect.hdf5 to S3
Added 0721RASGeneEffect.hdf5 to upload session 43a1dd7110e2434eb6fac60e5df37dd1
Uploading 0721RASGeneEffectScreenLevel.hdf5 to S3
Finished uploading 0721RASGeneEffectScreenLevel.hdf5 to S3
Completed uploading 0721RASGeneEffectScreenLevel.hdf5 to S3
Added 0721RASGeneEffectScreenLevel.hdf5 to upload session 43a1dd7110e2434eb6fac60e5df37dd1
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen07212025-0b0e/18



In [109]:
RASGeneEffectScreenLevel = tc.get(name=TAIGA_DATASET,file='RASGeneEffectScreenLevel')  # download_to_cache for raw
RASGeneEffect = RASGeneEffectScreenLevel

No dataset version provided. Using version 18.


# Create ScreenMap

In [110]:
RASScreenMap = tc.get(name=TAIGA_DATASET,file='RASScreenMap')


No dataset version provided. Using version 18.


In [111]:
RASScreenMap.columns

Index(['Unnamed: 0', 'Screen', 'CellLine', 'ScreenType', 'Treatment', 'Source',
       'CRISPR', 'CopyNumber', 'Expression', 'LOH', 'SpearmanAVANA_x',
       'UsedInChronos', 'NNMD', 'NNMD_zscore', 'MedianCommonEssentials',
       'MedianNonessentials', 'MeanCommonEssentials', 'MeanNonessentials',
       'Cas', 'Days', 'ModelID', 'PatientID', 'CellLineName',
       'StrippedCellLineName', 'CCLEName', 'DepmapModelType',
       'OncotreeLineage', 'OncotreePrimaryDisease', 'OncotreeSubtype',
       'OncotreeCode', 'SangerModelID', 'COSMICID', 'SpearmanAVANA_y'],
      dtype='object')

In [112]:
RASScreenMap['UsedInChronos'] = RASScreenMap['Screen'].isin(RASGeneEffect.index)

In [114]:
RASScreenMap.to_csv("RASScreenMap0721.csv", index=False)

## Uploading screenmap

In [115]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset(TAIGA_DATASET, 
                            "Uploading RASScreenMap", 
                            additions=[
    UploadedFile('RASScreenMap', 
                 local_path=os.path.join("RASScreenMap0721.csv"), 
                 format=LocalFormat.CSV_TABLE)
    ]
)

Created temporary upload session b6be7b33b92c49d7a120cb234020a9d3
Uploading RASScreenMap0721.csv to S3
Finished uploading RASScreenMap0721.csv to S3
Completed uploading RASScreenMap0721.csv to S3
Added RASScreenMap0721.csv to upload session b6be7b33b92c49d7a120cb234020a9d3
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen07212025-0b0e/19



# Split Parental screens

In [ ]:
# RASGeneEffect = tc.get(name=TAIGA_DATASET, file='RASGeneEffect')  
# RASScreenMap = tc.get(name=TAIGA_DATASET, file='RASScreenMap')

No dataset version provided. Using version 9.
No dataset version provided. Using version 9.


[                      ]  0% |   0.0 s/B |   0.0 B /   2.5 KiB | ETA:  --:--:--
[##################]100% |   2.2 MiB/s |   2.5 KiB /   2.5 KiB | ETA:  00:00:00


In [116]:
RASScreenMap

,Unnamed: 0,Screen,CellLine,ScreenType,Treatment,Source,CRISPR,CopyNumber,Expression,LOH,...,StrippedCellLineName,CCLEName,DepmapModelType,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,OncotreeCode,SangerModelID,COSMICID,SpearmanAVANA_y
0,0,PACADD188,PACADD188,Parental,NoTreatment,Sellers,True,True,True,True,...,PACADD188,PACADD188_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,None,NaN,0.462530
1,1,PANC0203_500x_D17,PANC0203,Parental,NoTreatment,Sellers,True,True,True,False,...,PANC0203,PANC0203_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,0.728071
2,2,PANC0203_500x_D13,PANC0203,Parental,NoTreatment,Sellers,True,True,True,False,...,PANC0203,PANC0203_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,0.644782
3,3,PANC0203_500x_D5,PANC0203,Parental,NoTreatment,Sellers,True,True,True,False,...,PANC0203,PANC0203_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,0.150563
4,4,PANC0203_500x_D9,PANC0203,Parental,NoTreatment,Sellers,True,True,True,False,...,PANC0203,PANC0203_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,SIDM01139,1298475.0,0.422733
5,5,PACADD165,PACADD165,Parental,NoTreatment,Sellers,True,True,True,False,...,PACADD165,PACADD165_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,None,NaN,0.732178
6,6,KP363T,KP363T,Parental,NoTreatment,Sellers,True,True,True,True,...,KP363T,KP363T_COLON,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,None,NaN,0.730205
7,7,SNU1079,SNU1079,Parental,NoTreatment,Sellers,True,True,True,True,...,SNU1079,SNU1079_BILIARY_TRACT,IHCH,Biliary Tract,Intraductal Papillary Neoplasm of the Bile Duct,Intrahepatic Cholangiocarcinoma,IHCH,SIDM00164,NaN,0.301804
8,8,COLO678,COLO678,Parental,NoTreatment,Sellers,True,True,True,True,...,COLO678,COLO678_LARGE_INTESTINE,COAD,Bowel,Colorectal Adenocarcinoma,Colon Adenocarcinoma,COAD,SIDM00957,910689.0,0.764347
9,9,SNU410,SNU410,Parental,NoTreatment,Sellers,True,True,True,False,...,SNU410,SNU410_PANCREAS,PAAD,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PAAD,None,NaN,0.834547


In [117]:
RASScreenMapFiltered =  RASScreenMap.loc[RASScreenMap['UsedInChronos'] & (RASScreenMap['ScreenType'] == 'Parental'),]
RASScreenMapFiltered = RASScreenMapFiltered.loc[RASScreenMapFiltered['Screen'] != 'NCIH747.WRS00022']

In [119]:
RASGeneEffectParental = RASGeneEffect.loc[RASScreenMapFiltered['Screen']]

In [121]:
RASGeneEffectParental.index = RASScreenMapFiltered['ModelID']

In [122]:
RASGeneEffectParental = RASGeneEffectParental.T


In [123]:
RASGeneEffectParental=RASGeneEffectParental.loc[:, RASGeneEffectParental.columns.notnull()]
RASGeneEffectParental #alina added. Is this meant to be?

ModelID,ACH-001382,ACH-000042,ACH-000042,ACH-001380,ACH-002669,ACH-000350,ACH-000517,ACH-000535,ACH-000377,ACH-000997,ACH-001039,ACH-000094,ACH-000915,ACH-000164,ACH-000042
ABCB7,-1.569134,-0.793802,-0.538895,-0.959826,-1.090039,-0.984268,-1.229478,-1.180169,-2.334467,-2.067564,-0.718061,-1.101443,-1.355939,-0.998866,-1.011734
ABCF1,-0.911991,-0.861029,-0.816415,-1.354044,-1.368010,-0.460997,-0.870666,-1.220149,-1.190085,-1.692507,-0.880274,-0.638445,-1.076866,-0.476316,-0.847090
ABL1,-0.103256,-0.158134,-0.130229,-0.352820,-0.280611,-0.187423,-0.114854,-0.224970,-0.088707,-0.031890,-0.062943,0.133116,0.005116,-0.078353,0.018231
ACTL7A,-0.047919,-0.100900,-0.186261,-0.449311,-0.206993,0.103760,-0.061827,-0.084473,-0.016812,0.100401,0.023210,-0.078443,-0.180341,0.035856,0.045486
ACTL7B,0.164574,0.095515,0.063563,0.187461,0.054511,-0.314502,-0.121758,0.011948,0.048151,0.068557,-0.048788,0.062730,0.051066,-0.160974,0.190189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAG,-0.012996,0.112183,0.134027,-0.016899,-0.038199,-0.093901,-0.030061,-0.173431,-0.042351,0.015064,-0.047827,-0.208105,-0.042486,0.135612,0.072276
YWHAG_YWHAZ,-0.185185,-0.060836,-0.060269,-0.166290,-0.145377,0.185038,0.219989,-0.301692,-0.183250,-0.471259,-0.859941,-1.069454,-0.453505,-0.648744,-0.085941
YWHAZ,-0.265480,-0.008762,0.100951,-0.148120,-0.114315,0.004712,0.060810,-0.191111,-0.092845,-0.141680,-0.437065,-0.486183,-0.315810,-0.342389,0.035667
YY1,-0.733314,-0.786611,-0.905282,-0.310542,-0.638440,-1.612199,-0.631666,-1.294156,-0.919524,-1.489052,-0.794764,-0.793323,-0.710198,-1.256823,-0.744145


In [124]:
RASGeneEffectParental=RASGeneEffectParental.loc[:,~RASGeneEffectParental.columns.duplicated()].copy()

In [125]:
RASGeneEffectParental.to_csv("0721RASGeneEffectParentalByModelID.csv")

In [126]:
RASGeneEffectParental

ModelID,ACH-001382,ACH-000042,ACH-001380,ACH-002669,ACH-000350,ACH-000517,ACH-000535,ACH-000377,ACH-000997,ACH-001039,ACH-000094,ACH-000915,ACH-000164
ABCB7,-1.569134,-0.793802,-0.959826,-1.090039,-0.984268,-1.229478,-1.180169,-2.334467,-2.067564,-0.718061,-1.101443,-1.355939,-0.998866
ABCF1,-0.911991,-0.861029,-1.354044,-1.368010,-0.460997,-0.870666,-1.220149,-1.190085,-1.692507,-0.880274,-0.638445,-1.076866,-0.476316
ABL1,-0.103256,-0.158134,-0.352820,-0.280611,-0.187423,-0.114854,-0.224970,-0.088707,-0.031890,-0.062943,0.133116,0.005116,-0.078353
ACTL7A,-0.047919,-0.100900,-0.449311,-0.206993,0.103760,-0.061827,-0.084473,-0.016812,0.100401,0.023210,-0.078443,-0.180341,0.035856
ACTL7B,0.164574,0.095515,0.187461,0.054511,-0.314502,-0.121758,0.011948,0.048151,0.068557,-0.048788,0.062730,0.051066,-0.160974
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YWHAG,-0.012996,0.112183,-0.016899,-0.038199,-0.093901,-0.030061,-0.173431,-0.042351,0.015064,-0.047827,-0.208105,-0.042486,0.135612
YWHAG_YWHAZ,-0.185185,-0.060836,-0.166290,-0.145377,0.185038,0.219989,-0.301692,-0.183250,-0.471259,-0.859941,-1.069454,-0.453505,-0.648744
YWHAZ,-0.265480,-0.008762,-0.148120,-0.114315,0.004712,0.060810,-0.191111,-0.092845,-0.141680,-0.437065,-0.486183,-0.315810,-0.342389
YY1,-0.733314,-0.786611,-0.310542,-0.638440,-1.612199,-0.631666,-1.294156,-0.919524,-1.489052,-0.794764,-0.793323,-0.710198,-1.256823


In [127]:
import os
from taigapy import create_taiga_client_v3
from taigapy.client_v3 import UploadedFile, LocalFormat, TaigaReference
tc = create_taiga_client_v3()
version = tc.update_dataset(TAIGA_DATASET, 
                            "Uploading RASGeneEffectParentalByModelID", 
                            additions=[
    UploadedFile('RASGeneEffectParentalByModelID', 
                 local_path=os.path.join("0721RASGeneEffectParentalByModelID.csv"), 
                 format=LocalFormat.CSV_MATRIX)
    ]
)

Created temporary upload session 40d7a1c0326e46299ef6ac1579655a09
Uploading 0721RASGeneEffectParentalByModelID.csv to S3
Finished uploading 0721RASGeneEffectParentalByModelID.csv to S3
Completed uploading 0721RASGeneEffectParentalByModelID.csv to S3
Added 0721RASGeneEffectParentalByModelID.csv to upload session 40d7a1c0326e46299ef6ac1579655a09
Dataset created. Access it directly with this url: https://cds.team/taiga/dataset/rasscreen07212025-0b0e/20

